#Quick Check that business event prediction will run on a Sample

In [1]:
#imports

# to load my data
#from google.colab import drive
import pandas as pd
import numpy as np

# to run the models
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

c:\Users\Chris\anaconda3\envs\p310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [33]:
# mount my google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load csv files of S&P 100 files
file_path_sp100 = 'sp100_news_2018_2023.csv'

df_sp100 = pd.read_csv(file_path_sp100)

C:\Users\Chris\AppData\Local\Temp\ipykernel_8324\2049233381.py:4: DtypeWarning: Columns (6,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sp100 = pd.read_csv(file_path_sp100)


In [3]:
# check that it looks right
df_sp100.head()

,Unnamed: 0.1,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,12025,12025.0,2023-12-16 22:00:00,My 6 Largest Portfolio Holdings Heading Into 2...,AAPL,https://www.nasdaq.com/articles/my-6-largest-p...,NaN,NaN,"After an absolute disaster of a year in 2022, ...",3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...
1,12026,12026.0,2023-12-16 22:00:00,Brokers Suggest Investing in Apple (AAPL): Rea...,AAPL,https://www.nasdaq.com/articles/brokers-sugges...,NaN,NaN,"When deciding whether to buy, sell, or hold a ...",Let's take a look at what these Wall Street he...,Click to get this free report Apple Inc. (AAPL...,Let's take a look at what these Wall Street he...,Brokerage Recommendation Trends for AAPL Let's...
2,12027,12027.0,2023-12-16 21:00:00,"Company News for Dec 19, 2023",AAPL,https://www.nasdaq.com/articles/company-news-f...,NaN,NaN,Shares of Apple Inc. AAPL lost 0.9% on China’s...,Shares of Apple Inc. AAPL lost 0.9% on China’s...,Click to get this free report Apple Inc. (AAPL...,Click to get this free report Apple Inc. (AAPL...,Click to get this free report Apple Inc. (AAPL...
3,12028,12028.0,2023-12-16 21:00:00,NVIDIA (NVDA) Up 243% YTD: Will It Carry Momen...,AAPL,https://www.nasdaq.com/articles/nvidia-nvda-up...,NaN,NaN,NVIDIA Corporation NVDA has witnessed a remark...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...
4,12029,12029.0,2023-12-16 21:00:00,"Pre-Market Most Active for Dec 19, 2023 : BMY,...",AAPL,https://www.nasdaq.com/articles/pre-market-mos...,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 10.1...,"Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,..."


In [36]:
# create a smaller sample so you can test things quickly
df_sp100_sample = df_sp100.sample(n=100)

## Load & Run the model

In [5]:
# FOR LORA
from peft import LoraModel, LoraConfig, TaskType, get_peft_model
# load your model and tokenizer
# model if finetuned, tokenizer is regular FinBERT
model = 'yiyanghkust/finbert-tone'
tokenizer = AutoTokenizer.from_pretrained(model)
saved_model_path = 'batch_16_best_model-lora.pt'
# load the model with correct number of labels
model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-pretrain",
    num_labels=12,
    ignore_mismatched_sizes=True
)
# Load the PEFT config and  weights
# Configure LoRA
lora_config = LoraConfig(
    r=8, # low rank factor
    lora_alpha=16, # scaling
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS
)

# add LoRA to the model
model_lora = get_peft_model(model, lora_config)

# move model to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lora.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30873, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [55]:
# NOT FOR LORA
# load your model and tokenizer
# model if finetuned, tokenizer is regular FinBERT
model = 'yiyanghkust/finbert-tone'
tokenizer = AutoTokenizer.from_pretrained(model)
saved_model_path = '/content/drive/MyDrive/DATASCI_266_NLP/EDT_Copy/checkpoint_finbert-edt_epoch_1.pth'
# load the model with correct number of labels
model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-pretrain",
    num_labels=12,
    ignore_mismatched_sizes=True
)
checkpoint = torch.load(saved_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

# move model to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

KeyboardInterrupt: 

In [6]:
# predict business events
def predict_business_events(text):
  """
  Predicts the business event category for a given text using a fine-tuned model.

    Args:
        text (str): The input text for which the business event prediction is to be made.

    Returns:
        int: The predicted business event class label (integer).
             The label corresponds to one of the pre-defined business event categories in the fine-tuned model.
  """
  model_lora.eval()
  # ensure the text is a string
  text = str(text)
  
  inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
  tokens = {key: val.to(device) for key, val in inputs.items()}
  
  model_lora.to(device)
  
  with torch.no_grad():
    outputs = model_lora(**tokens)
    logits = outputs.logits

    business_event = torch.argmax(logits, dim=1).item()
    return business_event

## Small Sample Pipeline

In [7]:
# predict business events
df_sp100_sample['business_event'] = df_sp100_sample['Article'].apply(lambda text: predict_business_events(text, model=model_lora))

NameError: name 'df_sp100_sample' is not defined

In [59]:
# see if it worked
df_sp100_sample['business_event'].value_counts()

,count
business_event,
4,75
11,20
5,4
8,1


In [61]:
 df_sp100_sample[df_sp100_sample['business_event'] == 4]

,Unnamed: 0.1,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,business_event
95369,803750,803750.0,2018-11-09 00:00:00,Utilities Select Sector SPDR Fund Experiences ...,DUK,https://www.nasdaq.com/articles/utilities-sele...,NaN,NaN,Looking today at week-over-week shares outstan...,Among the largest underlying components of XLU...,Among the largest underlying components of XLU...,Among the largest underlying components of XLU...,Among the largest underlying components of XLU...,4
211820,2098746,2098746.0,2019-03-08 00:00:00,"The Zacks Analyst Blog Highlights: Amazon, Ber...",TMO,https://www.nasdaq.com/articles/the-zacks-anal...,NaN,NaN,"For Immediate Release\nChicago, IL -March 8, 2...",Stocks recently featured in the blog include: ...,Click to get this free report Public Service E...,Click to get this free report Public Service E...,Stocks recently featured in the blog include: ...,4
136148,1393052,1393052.0,2023-05-25 00:00:00,2 Top Dividend Kings to Buy for the Long Haul,KO,https://www.nasdaq.com/articles/2-top-dividend...,NaN,NaN,The market had a difficult 2022. Last year rev...,Coca-Cola Despite rising costs putting pressur...,KO Dividend Yield data by YCharts But when cho...,Coca-Cola Despite rising costs putting pressur...,Coca-Cola Despite rising costs putting pressur...,4
121434,1188152,1188152.0,2021-09-21 00:00:00,AT&T plans multi-year effort to burnish its br...,GS,https://www.nasdaq.com/articles/att-plans-mult...,NaN,NaN,By Sheila Dang\nSept 21 (Reuters) - AT&T Inc T...,By Sheila Dang Sept 21 (Reuters) - AT&T Inc T....,By Sheila Dang Sept 21 (Reuters) - AT&T Inc T....,Speaking at Goldman Sachs' Communacopia confer...,By Sheila Dang Sept 21 (Reuters) - AT&T Inc T....,4
176094,1611286,1611286.0,2022-03-29 00:00:00,Why Nvidia Stock Jumped Today,NVDA,https://www.nasdaq.com/articles/why-nvidia-sto...,NaN,NaN,What happened\nShares of Nvidia (NASDAQ: NVDA)...,What happened Shares of Nvidia (NASDAQ: NVDA) ...,What happened Shares of Nvidia (NASDAQ: NVDA) ...,What happened Shares of Nvidia (NASDAQ: NVDA) ...,What happened Shares of Nvidia (NASDAQ: NVDA) ...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303897,4903420,NaN,2019-04-15,BlackRock Q1 2019 Earnings Preview,BLK,https://seekingalpha.com/news/3450762-blackroc...,Seeking Alpha,NaN,NaN,NaN,NaN,NaN,NaN,4
32854,191949,191949.0,2021-06-16 00:00:00,Allianz buys 10% stake in American Towers Europe,AMT,https://www.nasdaq.com/articles/allianz-buys-1...,NaN,NaN,"FRANKFURT, June 16 (Reuters) - German insurer ...","FRANKFURT, June 16 (Reuters) - German insurer ...","FRANKFURT, June 16 (Reuters) - German insurer ...","FRANKFURT, June 16 (Reuters) - German insurer ...","FRANKFURT, June 16 (Reuters) - German insurer ...",4
65559,624750,624750.0,2023-03-16 00:00:00,The Threat of E-Commerce Doesn't Scare This Un...,COST,https://www.nasdaq.com/articles/the-threat-of-...,NaN,NaN,With its trailing-12-month net revenue of $230...,With its trailing-12-month net revenue of $230...,With its trailing-12-month net revenue of $230...,With its trailing-12-month net revenue of $230...,* They just revealed what they believe are the...,4
126319,1355864,1355864.0,2023-05-16,INTC November 17th Options Begin Trading,INTC,https://www.nasdaq.com/articles/intc-november-...,NaN,NaN,Investors in Intel Corp (Symbol: INTC) saw new...,"Of course, a lot of upside could potentially b...",Below is a chart showing INTC's trailing twelv...,Below is a chart showing INTC's trailing twelv...,Below is a chart showing INTC's trailing twelv...,4


In [8]:
df_sp100['business_event'] = df_sp100['Article'].apply(predict_business_events)

c:\Users\Chris\anaconda3\envs\p310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [10]:
df_sp100.to_csv('sp100_news_2018_2023_events_added_LORA.csv')